# Datasets

A dataset is a list of (input, target) pairs that can be further split into training and testing lists.

Let's make an example network to use as demonstration. This network will compute whether the number of 1's in a set of 5 bits is odd.

In [1]:
from conx import Network, Layer

net = Network("Odd Network")
net.add(Layer("input", 5))
net.add(Layer("hidden", 10, activation="relu"))
net.add(Layer("output", 1, activation="sigmoid"))
net.connect()
net.compile(error="mse", optimizer="adam")
net.summary()

Using Theano backend.


Network Summary
---------------
Network name: Odd Network
    Layer name: 'input' (input)
        VShape: None
        Dropout: 0
        Connected to: ['hidden']
        Activation function: None
        Dropout percent: 0
    Layer name: 'hidden' (hidden)
        VShape: None
        Dropout: 0
        Connected to: ['output']
        Activation function: relu
        Dropout percent: 0
    Layer name: 'output' (output)
        VShape: None
        Dropout: 0
        Activation function: sigmoid
        Dropout percent: 0


## As a list of (input, target) pairs

The most straightforward method of adding input, target vectors to train on is to use a list of (input, target) pairs. First we define a function that takes a number and returns the bitwise representation of it:

In [2]:
def num2bin(i, bits=5):
    """
    Take a number and turn it into a list of bits (most significant first).
    """
    return [int(s) for s in (("0" * bits) + bin(i)[2:])[-bits:]]

In [3]:
num2bin(23)

[1, 0, 1, 1, 1]

Now we make a list of (input, target) pairs:

In [4]:
patterns = []

for i in range(2 ** 5):
    inputs = num2bin(i)
    targets = [int(sum(inputs) % 2 == 1.0)]
    patterns.append((inputs, targets))

Pair set 5 looks like:

In [5]:
patterns[5]

([0, 0, 1, 0, 1], [0])

We set the network to use this dataset:

In [6]:
patterns

[([0, 0, 0, 0, 0], [0]),
 ([0, 0, 0, 0, 1], [1]),
 ([0, 0, 0, 1, 0], [1]),
 ([0, 0, 0, 1, 1], [0]),
 ([0, 0, 1, 0, 0], [1]),
 ([0, 0, 1, 0, 1], [0]),
 ([0, 0, 1, 1, 0], [0]),
 ([0, 0, 1, 1, 1], [1]),
 ([0, 1, 0, 0, 0], [1]),
 ([0, 1, 0, 0, 1], [0]),
 ([0, 1, 0, 1, 0], [0]),
 ([0, 1, 0, 1, 1], [1]),
 ([0, 1, 1, 0, 0], [0]),
 ([0, 1, 1, 0, 1], [1]),
 ([0, 1, 1, 1, 0], [1]),
 ([0, 1, 1, 1, 1], [0]),
 ([1, 0, 0, 0, 0], [1]),
 ([1, 0, 0, 0, 1], [0]),
 ([1, 0, 0, 1, 0], [0]),
 ([1, 0, 0, 1, 1], [1]),
 ([1, 0, 1, 0, 0], [0]),
 ([1, 0, 1, 0, 1], [1]),
 ([1, 0, 1, 1, 0], [1]),
 ([1, 0, 1, 1, 1], [0]),
 ([1, 1, 0, 0, 0], [0]),
 ([1, 1, 0, 0, 1], [1]),
 ([1, 1, 0, 1, 0], [1]),
 ([1, 1, 0, 1, 1], [0]),
 ([1, 1, 1, 0, 0], [1]),
 ([1, 1, 1, 0, 1], [0]),
 ([1, 1, 1, 1, 0], [0]),
 ([1, 1, 1, 1, 1], [1])]

In [8]:
net.dataset.load(patterns)

In [9]:
net.dataset.summary()

Input Summary:
   count  : 32 (32 for training, 0 for testing)
   shape  : (5,)
   range  : (0.0, 1.0)
Target Summary:
   count  : 32 (32 for training, 0 for testing)
   shape  : (1,)
   range  : (0.0, 1.0)


## Dataset.add()

You can use the default `dataset` and add one pattern at a time. Consider the task of training a network to determine if the number of inputs is even (0) or odd (1). We could add inputs one at a time:

In [10]:
net.dataset.clear()

In [11]:
net.dataset.add([0, 0, 0, 0, 1], [1])
net.dataset.add([0, 0, 0, 1, 1], [0])
net.dataset.add([0, 0, 1, 0, 0], [1])

In [12]:
net.dataset.clear()

In [13]:
for i in range(2 ** 5):
    inputs = num2bin(i)
    targets = [int(sum(inputs) % 2 == 1.0)]
    net.dataset.add(inputs, targets)

In [14]:
net.dataset.summary()

Input Summary:
   count  : 32 (32 for training, 0 for testing)
   shape  : (5,)
   range  : (0.0, 1.0)
Target Summary:
   count  : 32 (32 for training, 0 for testing)
   shape  : (1,)
   range  : (0.0, 1.0)


In [15]:
net.dataset.inputs[13]

[0.0, 1.0, 1.0, 0.0, 1.0]

In [16]:
net.dataset.targets[13]

[1.0]

In [17]:
net.train(epochs=5000, accuracy=.75, tolerance=.2, report_rate=100)

Training...
Epoch #  100 | train error 0.25057 | train accuracy 0.56250 | validate% 0.00000
Epoch #  200 | train error 0.24481 | train accuracy 0.56250 | validate% 0.00000
Epoch #  300 | train error 0.24008 | train accuracy 0.50000 | validate% 0.00000
Epoch #  400 | train error 0.23402 | train accuracy 0.59375 | validate% 0.00000
Epoch #  500 | train error 0.22551 | train accuracy 0.71875 | validate% 0.00000
Epoch #  600 | train error 0.21735 | train accuracy 0.75000 | validate% 0.00000
Epoch #  700 | train error 0.20895 | train accuracy 0.78125 | validate% 0.00000
Epoch #  800 | train error 0.20028 | train accuracy 0.84375 | validate% 0.00000
Epoch #  900 | train error 0.19143 | train accuracy 0.87500 | validate% 0.00000
Epoch # 1000 | train error 0.18276 | train accuracy 0.87500 | validate% 0.00000
Epoch # 1100 | train error 0.17386 | train accuracy 0.87500 | validate% 0.00000
Epoch # 1200 | train error 0.16527 | train accuracy 0.87500 | validate% 0.00000
Epoch # 1300 | train error 0

In [18]:
net.test(tolerance=.2)

Testing on training dataset...
# | inputs | targets | outputs | result
---------------------------------------
0 | [0.00,0.00,0.00,0.00,0.00] | [0.00] | [0.02] | correct
1 | [0.00,0.00,0.00,0.00,1.00] | [1.00] | [0.93] | correct
2 | [0.00,0.00,0.00,1.00,0.00] | [1.00] | [0.83] | correct
3 | [0.00,0.00,0.00,1.00,1.00] | [0.00] | [0.14] | correct
4 | [0.00,0.00,1.00,0.00,0.00] | [1.00] | [0.95] | correct
5 | [0.00,0.00,1.00,0.00,1.00] | [0.00] | [0.08] | correct
6 | [0.00,0.00,1.00,1.00,0.00] | [0.00] | [0.13] | correct
7 | [0.00,0.00,1.00,1.00,1.00] | [1.00] | [0.99] | correct
8 | [0.00,1.00,0.00,0.00,0.00] | [1.00] | [0.81] | correct
9 | [0.00,1.00,0.00,0.00,1.00] | [0.00] | [0.04] | correct
10 | [0.00,1.00,0.00,1.00,0.00] | [0.00] | [0.24] | X
11 | [0.00,1.00,0.00,1.00,1.00] | [1.00] | [0.82] | correct
12 | [0.00,1.00,1.00,0.00,0.00] | [0.00] | [0.07] | correct
13 | [0.00,1.00,1.00,0.00,1.00] | [1.00] | [0.86] | correct
14 | [0.00,1.00,1.00,1.00,0.00] | [1.00] | [0.93] | correct
15 | 

## Dataset inputs and targets

Inputs and targets in the dataset are represented in the same format as given (as lists, or lists of lists). These formats are automattically converted into an internal format.

In [19]:
ds = net.dataset

In [20]:
ds.inputs[17]

[1.0, 0.0, 0.0, 0.0, 1.0]

In [22]:
net.test(tolerance=.2)

Testing on training dataset...
# | inputs | targets | outputs | result
---------------------------------------
0 | [0.00,0.00,0.00,0.00,0.00] | [0.00] | [0.02] | correct
1 | [0.00,0.00,0.00,0.00,1.00] | [1.00] | [0.93] | correct
2 | [0.00,0.00,0.00,1.00,0.00] | [1.00] | [0.83] | correct
3 | [0.00,0.00,0.00,1.00,1.00] | [0.00] | [0.14] | correct
4 | [0.00,0.00,1.00,0.00,0.00] | [1.00] | [0.95] | correct
5 | [0.00,0.00,1.00,0.00,1.00] | [0.00] | [0.08] | correct
6 | [0.00,0.00,1.00,1.00,0.00] | [0.00] | [0.13] | correct
7 | [0.00,0.00,1.00,1.00,1.00] | [1.00] | [0.99] | correct
8 | [0.00,1.00,0.00,0.00,0.00] | [1.00] | [0.81] | correct
9 | [0.00,1.00,0.00,0.00,1.00] | [0.00] | [0.04] | correct
10 | [0.00,1.00,0.00,1.00,0.00] | [0.00] | [0.24] | X
11 | [0.00,1.00,0.00,1.00,1.00] | [1.00] | [0.82] | correct
12 | [0.00,1.00,1.00,0.00,0.00] | [0.00] | [0.07] | correct
13 | [0.00,1.00,1.00,0.00,1.00] | [1.00] | [0.86] | correct
14 | [0.00,1.00,1.00,1.00,0.00] | [1.00] | [0.93] | correct
15 | 

To see/access the internal format, use the underscore before inputs or targets. This is a numpy array. conx is designed so that you need not have to use numpy for most network operations.

In [23]:
ds._inputs[17]

array([ 1.,  0.,  0.,  0.,  1.], dtype=float32)

## Built-in datasets

In [26]:
from conx import Dataset

In [27]:
net = Network("Test")

In [29]:
Dataset(net).get('mnist')

In [30]:
Dataset(net).get('cifar10')

In [31]:
Dataset(net).get('cifar100')

## Dataset operations

Dataset.split() will divide the dataset between training and testing sets. You can provide split an integer (to divide at a specific point), or a floating-point value, to divide by a percentage.

In [32]:
ds.split(20)

In [33]:
ds.split(.5)

In [34]:
ds.slice(10)

In [38]:
ds.shuffle()

In [37]:
ds.chop(5)

In [39]:
ds.summary()

Input Summary:
   count  : 5 (0 for training, 5 for testing)
   shape  : (5,)
   range  : (0.0, 1.0)
Target Summary:
   count  : 5 (0 for training, 5 for testing)
   shape  : (1,)
   range  : (0.0, 1.0)


### Additional operations

These functions are subject to change to an API which is more general:

In [40]:
ds.set_targets_from_inputs()

In [41]:
ds.set_inputs_from_targets()

In [43]:
# ds.set_targets_from_labels()

In [44]:
ds.inputs.shape()

(5,)

In [45]:
ds.inputs.reshape(0, (1, 5))

In [46]:
ds.inputs.shape()

(1, 5)

## Dataset direct manipulation

You can also set the internal format directly, given that it is in the correct format:

* use list of columns for multi-bank inputs or targets
* use np.array(vectors) for single-bank inputs or targets

In [49]:
import numpy as np

inputs = []
targets = []

for i in range(2 ** 5):
    v = num2bin(i)
    inputs.append(v)
    targets.append([int(sum(v) % 2 == 1.0)])

net = Network("Even?", 5, 2, 2, 1)
net.compile(error="mse", optimizer="adam")
net.dataset.load_direct(np.array(inputs), np.array(targets))

In [50]:
net.test(tolerance=.2)

Testing on training dataset...
# | inputs | targets | outputs | result
---------------------------------------
0 | [0,0,0,0,0] | [0] | [0.00] | correct
1 | [0,0,0,0,1] | [1] | [0.28] | X
2 | [0,0,0,1,0] | [1] | [-0.08] | X
3 | [0,0,0,1,1] | [0] | [0.20] | X
4 | [0,0,1,0,0] | [1] | [0.17] | X
5 | [0,0,1,0,1] | [0] | [0.45] | X
6 | [0,0,1,1,0] | [0] | [0.09] | correct
7 | [0,0,1,1,1] | [1] | [0.37] | X
8 | [0,1,0,0,0] | [1] | [-0.29] | X
9 | [0,1,0,0,1] | [0] | [-0.01] | correct
10 | [0,1,0,1,0] | [0] | [-0.37] | X
11 | [0,1,0,1,1] | [1] | [-0.09] | X
12 | [0,1,1,0,0] | [0] | [-0.13] | correct
13 | [0,1,1,0,1] | [1] | [0.15] | X
14 | [0,1,1,1,0] | [1] | [-0.20] | X
15 | [0,1,1,1,1] | [0] | [0.08] | correct
16 | [1,0,0,0,0] | [1] | [0.22] | X
17 | [1,0,0,0,1] | [0] | [0.50] | X
18 | [1,0,0,1,0] | [0] | [0.14] | correct
19 | [1,0,0,1,1] | [1] | [0.42] | X
20 | [1,0,1,0,0] | [0] | [0.38] | X
21 | [1,0,1,0,1] | [1] | [0.66] | X
22 | [1,0,1,1,0] | [1] | [0.31] | X
23 | [1,0,1,1,1] | [0] | [0.